# Loading data

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

df_product = pd.read_excel('/content/drive/MyDrive/Port/BI Pricing/Sample data BI Pricing.xlsx', sheet_name = 'dim_product_info')
df_sale_info = pd.read_excel('/content/drive/MyDrive/Port/BI Pricing/Sample data BI Pricing.xlsx', sheet_name = 'sale_info')
df_price_compe = pd.read_excel('/content/drive/MyDrive/Port/BI Pricing/Sample data BI Pricing.xlsx', sheet_name = 'price_competitor_info')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df_sale_info.merge(df_product, on= 'sku', how= 'left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_key                   900 non-null    datetime64[ns]
 1   order_id                   900 non-null    object        
 2   sku                        900 non-null    object        
 3   price                      900 non-null    float64       
 4   quantity                   900 non-null    int64         
 5   discount per qty           900 non-null    float64       
 6   shipping revenue           900 non-null    float64       
 7   shipping discount          900 non-null    float64       
 8   is_with_gift               900 non-null    int64         
 9   customer_id                900 non-null    object        
 10  customer shipping address  900 non-null    object        
 11  seller shipping address    900 non-null    object        
 12  product_

# Question 1

### 1a. Find out top 5 product from each sub cate report with most sales

In [ ]:
# Tạo cột revenue
df = df.copy()
df["revenue"] = df["price"] * df["quantity"]

# Revenue theo sub_cate_report + sku
df_revenue_by_sku = (df.groupby(["sub_cate_report", "sku"], as_index=False).agg(revenue=("revenue", "sum"), quantity= ("quantity", "sum")))

# TOP 5 mỗi sub_cate_report
top5_by_subcate = (df_revenue_by_sku.sort_values(["sub_cate_report", "revenue"], ascending=[True, False]).groupby("sub_cate_report", as_index=False).head(5).reset_index(drop=True))
top5_by_subcate

,sub_cate_report,sku,revenue,quantity
0,Accessories,SKU000153,192842.26,974
1,Accessories,SKU000657,142280.25,861
2,Accessories,SKU000315,135365.46,906
3,Accessories,SKU000327,127353.38,902
4,Accessories,SKU000551,120822.24,934
...,...,...,...,...
95,Women Clothing,SKU000718,163180.40,920
96,Women Clothing,SKU000159,152634.80,820
97,Women Clothing,SKU000054,151564.29,827
98,Women Clothing,SKU000505,151051.25,875


### 1b. Find out top 5 products in term of sales from each sub cate   report  that offering the best price in the market

Dữ liệu bao gồm:
- **df_sale_info**: ghi nhận giao dịch theo SKU by date, giá bán (price) và số lượng (quantity) thay đổi theo thời gian.
- **df_price_product**: ghi nhận giá tối thiểu của thị trường (min_price_competitor) theo SKU by date.
- **df_product**: cung cấp thông tin phân loại sản phẩm (sub_cate_report).

Do đặc thù thị trường, giá của cùng một SKU có thể cao hơn hoặc thấp hơn giá thị trường tại các thời điểm khác nhau. Vì vậy, cần một cách tiếp cận phù hợp để xác định doanh thu của sản phẩm, đồng thời đánh giá mức độ cạnh tranh về giá so với thị trường

**Hướng tiếp cận:**
- Doanh thu được tính trên toàn bộ giao dịch của từng SKU để phản ánh đúng hiệu quả bán hàng thực tế.
- Mức độ cạnh tranh về giá được đánh giá riêng bằng chỉ số **price index**, so sánh giá bán (price) trung bình của SKU với giá thị trường (min_price_competitor) theo thời gian.

=> Từ đó, các SKU có giá cạnh tranh được lựa chọn và xếp hạng top theo doanh thu trong từng sub-category.
- *price_index* <= 1: sản phẩm có giá cạnh tranh
- *price_index* > 1: sản phẩm đắt hơn giá thị trường



In [ ]:
#Competitor price trung bình trên mỗi SKU
df_price_compe= df_price_compe.groupby('sku').agg(avg_price_compe=('min_price_competitor', 'mean')).reset_index()
df_price_compe

,sku,avg_price_compe
0,SKU000001,126.09
1,SKU000002,94.87
2,SKU000003,169.68
3,SKU000004,180.37
4,SKU000005,181.22
...,...,...
695,SKU000696,113.52
696,SKU000697,174.18
697,SKU000698,100.50
698,SKU000699,89.89


In [ ]:
# Revenue theo sub_cate_report + sku
df_revenue_by_sku

# Selling price trung bình trên mỗi SKU
df_revenue_by_sku['avg_price']= df_revenue_by_sku['revenue']/df_revenue_by_sku['quantity']
df_revenue_by_sku

#Đánh giá mức độ cạnh tranh giá bằng chỉ số price_index
df_price_index = df_revenue_by_sku.merge(df_price_compe, on= 'sku', how= 'left')
df_price_index['price_index']= df_price_index['avg_price']/df_price_index['avg_price_compe']
df_price_index

#Lọc top 5 sản phẩm theo từng category có daonh thu cao nhất và có giá tốt nhất trên thị trường
result_best_price = df_price_index[df_price_index['price_index']<= 1].sort_values(["sub_cate_report", "revenue"], ascending=[True, False]).groupby('sub_cate_report', as_index= False).head(5).reset_index(drop= True)
result_best_price

,sub_cate_report,sku,revenue,quantity,avg_price,avg_price_compe,price_index
0,Accessories,SKU000657,142280.25,861,165.25,175.54,0.941381
1,Accessories,SKU000551,120822.24,934,129.36,137.12,0.943407
2,Accessories,SKU000043,104229.00,555,187.80,198.43,0.946429
3,Accessories,SKU000339,94443.21,747,126.43,127.09,0.994807
4,Accessories,SKU000158,78039.88,509,153.32,166.07,0.923225
...,...,...,...,...,...,...,...
95,Women Clothing,SKU000270,126109.55,985,128.03,135.38,0.945708
96,Women Clothing,SKU000031,99576.00,922,108.00,108.72,0.993377
97,Women Clothing,SKU000430,87517.22,778,112.49,121.49,0.925920
98,Women Clothing,SKU000584,69622.71,951,73.21,75.71,0.966979


### 1c. Find out top 10 products from each sub cate report with high sales that are more expensive than competiors

In [ ]:
#Lọc top 10 sản phẩm theo từng category có doanh thu cao nhất và có mức giá cao hơn so với thị trường
result_high_price = df_price_index[df_price_index['price_index']> 1].sort_values(["sub_cate_report", "revenue"], ascending=[True, False]).groupby('sub_cate_report', as_index= False).head(10).reset_index(drop= True)
result_high_price

,sub_cate_report,sku,revenue,quantity,avg_price,avg_price_compe,price_index
0,Accessories,SKU000153,192842.26,974,197.99,193.88,1.021199
1,Accessories,SKU000315,135365.46,906,149.41,148.35,1.007145
2,Accessories,SKU000327,127353.38,902,141.19,132.16,1.068326
3,Accessories,SKU000144,111919.50,950,117.81,109.35,1.077366
4,Accessories,SKU000240,103260.90,790,130.71,122.76,1.064761
...,...,...,...,...,...,...,...
195,Women Clothing,SKU000246,101656.96,823,123.52,117.55,1.050787
196,Women Clothing,SKU000459,73913.28,952,77.64,73.88,1.050893
197,Women Clothing,SKU000195,64987.60,380,171.02,160.45,1.065877
198,Women Clothing,SKU000585,58492.92,318,183.94,180.71,1.017874


### 1d. Propose approach to group products

Đề xuất phân nhóm sản phẩm (SKU) dựa trên 3 tiêu chí:
- **Vertical** (Ngành hàng): Mỗi SKU được gán vào một vertical (ví dụ: Electronics, FMCG, Fashion…) phản ánh đặc thù của từng ngành hàng. Việc phân tích và so sánh được thực hiện trong cùng vertical để đảm bảo tính hợp lý.
- **Price band** (Phân khúc giá): SKU được phân loại thành các price band (***Low / Mid / High***) dựa trên avg_price. Price band phân loại sản phẩm theo mức giá, hỗ trợ các quyết định về pricing, promotion và định vị sản phẩm.
- **Sale tier** (Hiệu quả bán hàng): Sales tier được xác định bằng phương pháp percentile dựa trên tổng doanh thu (revenue) trong từng category. Cách này cho phép phân loại SKU thành các nhóm như ***Best seller, Normal mover*** và ***Slow mover***, phản ánh hiệu quả bán hàng một cách tương đối đối với tất cả ngành hàng, tránh phụ thuộc vào ngưỡng cố định.

Khi kết hợp vertical + price band + sales tier, mỗi SKU sẽ được gán vào một product group, cách phân nhóm này giúp:
- Phân tích hiệu quả sản phẩm theo nhiều góc nhìn
- Dễ dàng lọc, so sánh và theo dõi trên dashboard BI
- Hỗ trợ ra quyết định về pricing, assortment và chiến lược kinh doanh

In [ ]:
# PRICE BAND (dựa trên avg_price)
df_price_index["avg_price"] = pd.to_numeric(df_price_index["avg_price"], errors="coerce")

df_price_index["price_band"] = np.select(
    [
        df_price_index["avg_price"] <= 100,
        (df_price_index["avg_price"] > 100) & (df_price_index["avg_price"] <= 1000000),
        df_price_index["avg_price"] > 1000000
    ],
    ["Low price", "Mid price", "High price"],
    default=None
)

# SALES TIER theo percentile revenue trong từng sub_cate_report
df_price_index["revenue_pct"] = (
    df_price_index.groupby("sub_cate_report")["revenue"]
                  .rank(method="first", ascending=False, pct=True)
)
df_price_index["revenue_pct"] = pd.to_numeric(df_price_index["revenue_pct"], errors="coerce")

df_price_index["sales_tier"] = np.select(
    [
        df_price_index["revenue_pct"] <= 0.2,
        (df_price_index["revenue_pct"] > 0.2) & (df_price_index["revenue_pct"] <= 0.7),
        df_price_index["revenue_pct"] > 0.7
    ],
    ["Best seller", "Normal mover", "Slow mover"],
    default=None
)

df_price_index.drop(columns="revenue_pct", inplace=True)

df_price_index


,sub_cate_report,sku,revenue,quantity,avg_price,avg_price_compe,price_index,price_band,sales_tier
0,Accessories,SKU000019,8273.56,46,179.86,185.37,0.970276,Mid price,Slow mover
1,Accessories,SKU000032,6458.40,240,26.91,26.92,0.999629,Low price,Slow mover
2,Accessories,SKU000043,104229.00,555,187.80,198.43,0.946429,Mid price,Best seller
3,Accessories,SKU000062,11778.71,97,121.43,130.99,0.927017,Mid price,Slow mover
4,Accessories,SKU000077,71508.00,808,88.50,79.91,1.107496,Low price,Normal mover
...,...,...,...,...,...,...,...,...,...
795,Women Clothing,SKU000718,163180.40,920,177.37,NaN,NaN,Mid price,Best seller
796,Women Clothing,SKU000734,70050.40,980,71.48,NaN,NaN,Low price,Normal mover
797,Women Clothing,SKU000738,3657.22,346,10.57,NaN,NaN,Low price,Slow mover
798,Women Clothing,SKU000772,5205.20,308,16.90,NaN,NaN,Low price,Slow mover


# Question 2

### 2a. Write a function that will be able to return the smallest positive integer missing from an unsorted array

In [ ]:
def smallest_missing_positive(nums):
    # Lọc các số nguyên dương
    positives = {x for x in nums if isinstance(x, int) and x > 0}

    # Tìm số dương nhỏ nhất bị thiếu, bắt đầu từ 1
    missing = 1
    while missing in positives:
        missing += 1
    return missing


a = [2, 3, -7, 6, 8, 1, -10, 15, -35]
smallest_missing_positive(a)

4

### 2b. Write a function that will be able to return the Nth element from the following sequence

In [ ]:
b = [0, 3, 3, 6, 9, 15, 24, 39, 63, 102, 165]

def number(x):
  if x < 1 or x > len(b):
     raise ValueError("x nằm ngoài dãy số")
  return b[x-1]

number(6)

15

### 2c. Write a function that will be able to remove duplicates from a given array.

Output requirements:
- Loại bỏ khoảng trắng
- Không phân biệt chữ hoa, chữ thường khi xác định trùng lặp
- Chuẩn hóa format chữ (Viết hoa chữ cái đầu tiên)
- Liệt kê các giá trị không trùng lặp và sắp xếp theo alphabet

In [ ]:
def remove_duplicates(array):
    seen = {}

    for item in array:
        # Loại bỏ khoảng trắng dư + khoảng trắng giữa từ
        cleaned = " ".join(item.strip().split())

        # Dùng lowercase để check trùng (case-insensitive)
        key = cleaned.lower()

        # Chuẩn hoá format chữ (Title Case)
        if key not in seen:
            seen[key] = cleaned.title()

    # Sort alphabetically
    return sorted(seen.values())

c= ['Panasonic','pensonic','panasonic ', 'Haier', 'electrolux    ', ' Electrolux', "Samsung", "samsung  "]

print(remove_duplicates(c))


['Electrolux', 'Haier', 'Panasonic', 'Pensonic', 'Samsung']


# Question 3

### Maximize profit on each product line

By grouping products by vertical, price band, and sales tier, different pricing and promotion strategies can be applied to each group.
- High-performing and high-price products focus on margin protection, while slow-moving products are optimized through discounting, bundling, or assortment rationalization.
- This segmented approach enables targeted actions to maximize overall profit rather than applying a single pricing strategy across all products.

Các đề xuất trên được xây dựng dựa trên dữ liệu giá và doanh thu, và chưa xem xét các yếu tố như mức độ hiển thị, đánh giá người dùng, nội dung và hình ảnh sản phẩm.
1. High price – Best seller
- Đặc điểm:
Giá cao, doanh thu lớn
Thường là sản phẩm chủ lực
- Chiến lược:
Hạn chế discount sâu để bảo vệ biên lợi nhuận
Tối ưu chi phí logistics và marketing
Áp dụng dynamic pricing dựa trên giá competitor
- Mục tiêu
➡️ Giữ doanh thu cao + cải thiện margin

Đây là các sản phẩm chủ lực với doanh thu lớn và biên lợi nhuận cao. Chiến lược tập trung vào bảo vệ margin bằng cách hạn chế giảm giá sâu, đồng thời tối ưu chi phí logistics và marketing. Dynamic pricing có thể được áp dụng dựa trên biến động giá đối thủ để vừa giữ doanh thu vừa cải thiện lợi nhuận.

🔹 2. High price – Slow mover
- Đặc điểm:
Giá cao nhưng bán chậm
Dễ tồn kho, chiếm vốn
- Chiến lược:
Xem xét giảm giá có kiểm soát
Bundle với sản phẩm bán chạy
Đánh giá lại assortment (có nên tiếp tục bán hay không)
- Mục tiêu:➡️ Giải phóng tồn kho và giảm chi phí vốn

Nhóm sản phẩm này có giá trị cao nhưng bán chậm, dễ phát sinh tồn kho và chiếm vốn. Chiến lược phù hợp là giảm giá có kiểm soát, kết hợp bundle với sản phẩm bán chạy hoặc đánh giá lại danh mục để quyết định tiếp tục hay loại bỏ. Mục tiêu là giải phóng tồn kho và giảm chi phí vốn.

🔹 3. Mid price – Best seller
- Đặc điểm:
Cân bằng giữa volume và margin
Thường có độ co giãn giá tốt
- Chiến lược:
Tối ưu giá để tăng volume mà không ảnh hưởng nhiều đến margin
Ưu tiên chạy promotion ngắn hạn
Upsell sang sản phẩm high price
- Mục tiêu
➡️ Tăng tổng lợi nhuận tuyệt đối

Các sản phẩm này cân bằng tốt giữa volume và margin, thường có độ co giãn giá cao. Doanh nghiệp có thể tối ưu giá hoặc chạy các chương trình khuyến mãi ngắn hạn để tăng sản lượng bán, đồng thời tận dụng nhóm này để upsell sang các sản phẩm high price. Mục tiêu là tối đa hóa tổng lợi nhuận.

🔹 4. Low price – Best seller
- Đặc điểm:
Bán rất chạy nhưng biên lợi nhuận thấp
- Chiến lược:
Hạn chế giảm giá thêm
Kiểm soát chặt chi phí fulfilment
Dùng làm sản phẩm kéo traffic (loss leader)
- Mục tiêu
➡️ Tối ưu chi phí và tăng giá trị giỏ hàng

Low-price best sellers often have thin margins, so profit optimization focuses on fulfillment cost control rather than further price reduction.
These products can also be used as loss leaders to attract traffic and drive sales of higher-margin items.

🔹 5. Low price – Slow mover
- Đặc điểm:
Doanh thu thấp, margin thấp
Ít đóng góp lợi nhuận
- Chiến lược:
Giảm độ ưu tiên hiển thị
Thanh lý hoặc loại khỏi danh mục
Chuyển sang bán theo combo
- Mục tiêu:
➡️ Cắt lỗ và tinh gọn danh mục
Đây là nhóm đóng góp thấp cả về doanh thu và lợi nhuận. Chiến lược nên tập trung vào giảm ưu tiên hiển thị, thanh lý hàng tồn hoặc loại khỏi danh mục, đồng thời cân nhắc bán theo combo để hạn chế thua lỗ. Mục tiêu là tinh gọn danh mục và cắt giảm các SKU kém hiệu quả.

🔹 Best seller + Price index ≤ 1
- Giá đang cạnh tranh
Có thể tăng nhẹ giá để cải thiện margin mà không ảnh hưởng nhiều đến doanh thu

🔹 Slow mover + Price index > 1
- Giá cao hơn thị trường, bán chậm
- Giảm giá có kiểm soát để kích cầu

🔹 Low price – Best seller
- Bán rất chạy, biên mỏng
- Giữ giá ổn định, tối ưu chi phí thay vì giảm giá thêm